# maude-magic

> Execute a maude`s session in Jupyter Lab

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- See persistent Shells.ipynb  

## The Maude Interpreter class

In [1]:
#| default_exp maude-magic

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
import os
import pexpect
import time

In [4]:
#| export
timeout = 3

class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        self.debug,self.timeout = debug,timeout
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
        # the expawned process:
        self.sh = pexpect.spawn('maude --no-advise', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        """ We syncrhonize with maude shell requesting a pwd.""" 
        self.sh.sendline('pwd')
        self.sh.expect('pwd.*Maude> ')
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')
             
    # Cmmand processing
    def __call__(self,command,timeout=timeout):
        """ Adds '\n' to command if it don't terminate with it. """
        # Don't terminate session by command
        if self.debug: print(f"Original Command = {repr(command)}")
        if command == 'quit .': return ''
        # strip command before send
        if self.debug: print(f"<--{repr(command.strip())}")
        if command[-1] != '\n' : command += '\n'
        if self.debug: print(f"Sent Command = {repr(command)}")
        self.sh.send(command)
        self._sync()
        if self.debug: print(f"-->{repr(self.sh.before)+repr(self.sh.after)}")
        response = self.sh.before
        # filter response
        return response
        

Creating maude interpreter:

In [5]:
maude=MaudeInterpreter(debug=False)

Load a maude module. Show it and make some reduction:

In [6]:
print(maude('load SIMPLE-NAT .'))

load SIMPLE-NAT .
Maude> 


In [7]:
print(maude('show module .'))
print(maude('red s s zero .'))

show module .
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> 
red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> 


Skip `quit` command:

In [8]:
maude('quit .')

''

Closing maude session on object destroy:

In [9]:
del maude 

## The maude Magic Class

In [10]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [11]:
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        super(MaudeMagics,self).__init__(MaudeInterpreter(debug=False)) 
        # print("On Construntor:"+str(type(self.shell)))
        self.line_counter=0
        
    def prepare_request(self,cell_contents:str)->str:
        """ - Removes leading and trailing empty lines from cell
            - Removes leading and trailing empty lines from each line
            - Terminates each line with \r\n
            - Count efective lines sent to maude shell.
        """
        # print(f"cell_contents at maude()={cell_contents}")
        cell_contents=cell_contents.strip()
        self.before_count =self.line_counter
        request = ""
        for cell_line in cell_contents.split('\n'):
            request+=cell_line.strip()+'\n'
            self.line_counter+=1
        # Maude shell will add trailing \n
        return request
    
    __prepare_request=prepare_request 

    def prepare_response(self,shell_response:str)->str:
        """ Ads the count of sent lines at the header of respone,
            to ease sintax error location.
        """    
        response =  f"{self.before_count} (lines sent before.)\n"
        response += ("--------------------------------------------\n")
        for line in shell_response.split('\n'):
            line =  line.strip()
            if line[0]=='>':
                self.before_count += 1
                line = f"{self.before_count:3} {line}"
            response += line + '\n'
        return response
    
    __prepare_response = prepare_response     
           
    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            return self.shell(line)
        else:
            # print(f"cell at maude()={cell}")
            print(self.prepare_response(self.shell(self.prepare_request(cell))))

# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [12]:
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [13]:
result = %maude load SIMPLE-NAT . 
result
#assert result == 'load SIMPLE-NAT .\n\r\rMaude> '

'load SIMPLE-NAT .\n\r\rMaude> '

In [14]:
%%maude
show module .  

0 (lines sent before.)
--------------------------------------------
show module .
fmod SIMPLE-NAT is
sort Nat .
op zero : -> Nat .
op s_ : Nat -> Nat .
op _+_ : Nat Nat -> Nat .
vars M N : Nat .
eq zero + N = N .
eq s N + M = s (N + M) .
endfm
Maude>



In [15]:
%%maude
red s s zero .    

1 (lines sent before.)
--------------------------------------------
red s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude>



## Use cases as test

### RDF Dataset

fth RDF-DATASET is 

    sort RDF-Dataset .

    op <_,_> : UnnamedGraph NamedGraph* -> RDF-Dataset [ctor] .    
    op NamedGraph : ResourceIdentifier Graph odeIdentifier    
        
        
        
        sort RDF-Document .

    op serializes : RDF-Dataset -> JSON-LD-Document .
    op serializes : Property -> IRI . 

    
    sorts NamedGraph NamedGraph+ NamedGraph* .

    
        


    op serializes : Property -> ResourceIdentifier .     
    
    op namedGraph : NodeIdentifier Graph -> 
    op _-[_]->_ : SourceNode IRI Node -> Arc [ctor] . 


    IRI BlankNodeId < SourceNodeId < NodeId Literal < NodeId .   
        

### JSON Syntax

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#uses-of-json-objects)

In [16]:
%%maude 
fmod JSON-SORTS is
    sort Json 
    sorts  Object Array .
    subsorts Object Array < Json .    
    sort Entry .    
    op null : -> Json [ctor] .    
endfm         
  
view Json  from TRIV to JSON-SORTS is sort Elt to Json . endv
view Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv

fmod JSON-ARRAY is
    protecting LIST{Json} * ( op (__) to (_,_) ) .
    op [_] : List{Json} -> Array .
    eq [nil] = null .
    op [] : -> Array [ctor] .
    eq [] = null .    
endfm

fmod JSON-OBJECT is
    protecting SET{Entry} .
    op {_} : Set{Entry} -> Object .
    eq {empty} = null .    
    
    sorts Key Value .
    op _:_ : Key Value  -> Entry [ctor] .
endfm        
             
fmod JSON is
    extending JSON-SORTS .
    protecting JSON-ARRAY .
    protecting JSON-OBJECT .        
    protecting STRING .

    subsort String  < Key Json .
    subsort Json < Value .
        
endfm 

2 (lines sent before.)
--------------------------------------------
fmod JSON-SORTS is
  3 > sort Json
  4 > sorts  Object Array .
  5 > subsorts Object Array < Json .
  6 > sort Entry .
  7 > op null : -> Json [ctor] .
  8 > endfm
Maude>
  9 > view Json  from TRIV to JSON-SORTS is sort Elt to Json . endv
Maude> view Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv
Maude>
 10 > fmod JSON-ARRAY is
 11 > protecting LIST{Json} * ( op (__) to (_,_) ) .
 12 > op [_] : List{Json} -> Array .
 13 > eq [nil] = null .
 14 > op [] : -> Array [ctor] .
 15 > eq [] = null .
 16 > endfm
Maude>
 17 > fmod JSON-OBJECT is
 18 > protecting SET{Entry} .
 19 > op {_} : Set{Entry} -> Object .
 20 > eq {empty} = null .
 21 >
 22 > sorts Key Value .
 23 > op _:_ : Key Value  -> Entry [ctor] .
 24 > endfm
Maude>
 25 > fmod JSON is
 26 > extending JSON-SORTS .
 27 > protecting JSON-ARRAY .
 28 > protecting JSON-OBJECT .
 29 > protecting STRING .
 30 >
 31 > subsort String  < Key Json .
 32 > subsort Js

In [17]:
result = %maude red "Juan" : "Perico" .
print(result)    
assert 'Entry: "Juan" : "Perico"' in result      

red "Juan" : "Perico" .
reduce in JSON : "Juan" : "Perico" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Entry: "Juan" : "Perico"
Maude> 


In [18]:
result = %maude red \
{\
  "name" : "Manu Sporny" ,\
  "homepage" : "http://manu.sporny.org/" ,\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
assert "Object:" in result    

In [19]:
result = %maude red ["Juan","Perico","Andres"] .
print(result)    
assert 'Array: ["Juan", "Perico", "Andres"]' in result


red ["Juan","Perico","Andres"] .
reduce in JSON : ["Juan", "Perico", "Andres"] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Array: ["Juan", "Perico", "Andres"]
Maude> 


In [20]:
result = %maude red ["Juan",["Perico","Andres"]] .
print(result)    
assert 'Array: ["Juan", ["Perico", "Andres"]]' in result 

red ["Juan",["Perico","Andres"]] .
reduce in JSON : ["Juan", ["Perico", "Andres"]] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Array: ["Juan", ["Perico", "Andres"]]
Maude> 


In [21]:
result = %maude red ["Juan", {"Perico" : "Andres"} ] .
print(result)    
assert 'Array: ["Juan", {"Perico" : "Andres"}]' in result 

red ["Juan", {"Perico" : "Andres"} ] .
reduce in JSON : ["Juan", {"Perico" : "Andres"}] .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Array: ["Juan", {"Perico" : "Andres"}]
Maude> 


In [22]:
result = %maude red {"Juan" : {"Perico" : "Andres"} } .
print(result)    
assert 'Object: {"Juan" : {"Perico" : "Andres"}}' in result 

red {"Juan" : {"Perico" : "Andres"} } .
reduce in JSON : {"Juan" : {"Perico" : "Andres"}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Object: {"Juan" : {"Perico" : "Andres"}}
Maude> 


In [23]:
result = %maude red {"Juan" : ["Perico", "Andres"]} .
print(result)    
assert 'Object: {"Juan" : ["Perico", "Andres"]}' in result 

red {"Juan" : ["Perico", "Andres"]} .
reduce in JSON : {"Juan" : ["Perico", "Andres"]} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Object: {"Juan" : ["Perico", "Andres"]}
Maude> 


In [24]:
result = %maude red \
{\
  "name" : "Manu Sporny",\
  "homepage" : "http://manu.sporny.org/",\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
print(result)
assert 'Object:' in result

red  {   "name" : "Manu Sporny",   "homepage" : "http://manu.sporny.org/",   "image" : "http://manu.sporny.org/images/manu.png" } .
reduce in JSON : {("homepage" : "http://manu.sporny.org/", "image" :
    "http://manu.sporny.org/images/manu.png"), "name" : "Manu Sporny"} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Object: {"homepage" : "http://manu.sporny.org/", "image" :
    "http://manu.sporny.org/images/manu.png", "name" : "Manu Sporny"}
Maude> 


In [25]:
result = %maude red \
{\
  "@context" : {\
    "name" : "http://schema.org/name",\
    "image" : {\
      "@id" : "http://schema.org/image",\
      "@type" : "@id"\
    },\
    "homepage" : {\
      "@id" : "http://schema.org/url",\
      "@type" : "@id"\
    }\
  }\
} .
print(result)
assert 'Object:' in result

red  {   "@context" : {     "name" : "http://schema.org/name",     "image" : {       "@id" : "http://schema.org/image",       "@type" : "@id"     },     "homepage" : {       "@id" : "http://schema.org/url",       "@type" : "@id"     }   } } .
reduce in JSON : {"@context" : {("homepage" : {"@id" : "http://schema.org/url",
    "@type" : "@id"}, "image" : {"@id" : "http://schema.org/image", "@type" :
    "@id"}), "name" : "http://schema.org/name"}} .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Object: {"@context" : {"homepage" : {"@id" : "http://schema.org/url",
    "@type" : "@id"}, "image" : {"@id" : "http://schema.org/image", "@type" :
    "@id"}, "name" : "http://schema.org/name"}}
Maude> 


### IRIS
[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#terms)

In [26]:
%%maude
fmod IRI is
protecting STRING .
sort IRI .
subsort IRI < String .
cmb S:String : IRI if find(S:String,":",0) :: NzNat .
endfm    


40 (lines sent before.)
--------------------------------------------
fmod IRI is
 41 > protecting STRING .
 42 > sort IRI .
 43 > subsort IRI < String .
 44 > cmb S:String : IRI if find(S:String,":",0) :: NzNat .
 45 > endfm
Maude>



In [27]:
result = %maude red "Http://perico" .
assert 'IRI: "Http://perico"' in result   

### Keywords

In [28]:
%%maude
fmod KEYWORD is
    protecting STRING .
    sort Keyword .
    subsort Keyword < String .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@included" : Keyword .
    mb "@graph"    : Keyword .
    mb "@nest"     : Keyword .
    mb "@type"     : Keyword .
    mb "@reverse"  : Keyword .
    mb "@index"    : Keyword .
endfm    


46 (lines sent before.)
--------------------------------------------
fmod KEYWORD is
 47 > protecting STRING .
 48 > sort Keyword .
 49 > subsort Keyword < String .
 50 > mb "@context"  : Keyword .
 51 > mb "@id"       : Keyword .
 52 > mb "@context"  : Keyword .
 53 > mb "@id"       : Keyword .
 54 > mb "@included" : Keyword .
 55 > mb "@graph"    : Keyword .
 56 > mb "@nest"     : Keyword .
 57 > mb "@type"     : Keyword .
 58 > mb "@reverse"  : Keyword .
 59 > mb "@index"    : Keyword .
 60 > endfm
Maude>



In [29]:
result = %maude red "@context" . 
print(result)
assert 'Keyword: "@context"' in result     

red "@context" .
reduce in KEYWORD : "@context" .
rewrites: 1 in 0ms cpu (0ms real) (~ rewrites/second)
result Keyword: "@context"
Maude> 


### Context

In [30]:
%%maude 
fmod CONTEXT is
    protecting IRI .
    extending JSON *( sort Object to Json-Object ) .
        
    sorts TermDef Context .
    subsort TermDef < Entry .
    subsort Context < Json-Object .
        
    sorts ContextKey ContextValue .
    subsort String < ContextKey < Key .
    subsorts IRI Json-Object < ContextValue < Value .

    mb (K:ContextKey : V:ContextValue) : TermDef .
    var ... : Set{Entry} . 
    cmb { T:TermDef , ...} : Context if {...} :: Context .
    mb null : Context .
        
endfm  


61 (lines sent before.)
--------------------------------------------
fmod CONTEXT is
 62 > protecting IRI .
 63 > extending JSON *( sort Object to Json-Object ) .
 64 >
 65 > sorts TermDef Context .
 66 > subsort TermDef < Entry .
 67 > subsort Context < Json-Object .
 68 >
 69 > sorts ContextKey ContextValue .
 70 > subsort String < ContextKey < Key .
 71 > subsorts IRI Json-Object < ContextValue < Value .
 72 >
 73 > mb (K:ContextKey : V:ContextValue) : TermDef .
 74 > var ... : Set{Entry} .
 75 > cmb { T:TermDef , ...} : Context if {...} :: Context .
 76 > mb null : Context .
 77 >
 78 > endfm
Maude>



In [31]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
    "name" : "http://schema.org/name"\
    ***(This means that 'name' is shorthand for 'http://schema.org/name')\
.
print(result)
assert('result TermDef:' in result) 

red     "name" : "http://schema.org/name"     ***(This means that 'name' is shorthand for 'http://schema.org/name') .
reduce in CONTEXT : "name" : "http://schema.org/name" .
rewrites: 6 in 0ms cpu (0ms real) (~ rewrites/second)
result TermDef: "name" : "http://schema.org/name"
Maude> 


In [32]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
{\
    "name" : "http://schema.org/name"\
     ***(This means that 'name' is shorthand for 'http://schema.org/name')\
} .
print(result)
assert('result Context:' in result) 

red {     "name" : "http://schema.org/name"      ***(This means that 'name' is shorthand for 'http://schema.org/name') } .
reduce in CONTEXT : {"name" : "http://schema.org/name"} .
rewrites: 10 in 0ms cpu (0ms real) (~ rewrites/second)
result Context: {"name" : "http://schema.org/name"}
Maude> 


In [33]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red { empty } .
print(result)
assert('result Context:' in result) 

red { empty } .
reduce in CONTEXT : {empty} .
rewrites: 2 in 0ms cpu (0ms real) (~ rewrites/second)
result Context: null
Maude> 


In [34]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
{\
    "@context" : {\
        "name" : "http://schema.org/name",\
        ***(  This means that 'name' is shorthand for 'http://schema.org/name')\
        "image" : {\
            "@id" : "http://schema.org/image",\
            ***(  This means that 'image' is shorthand for 'http://schema.org/image')\
            "@type" : "@id"\
            ***( This means that a string value associated with 'image')\
            ***( should be interpreted as an identifier that is an IRI)\
            },\
        "homepage" : {\
            "@id" : "http://schema.org/url",\
            ***( This means that 'homepage' is shorthand for 'http://schema.org/url')\
            "@type" : "@id"\
            ***(  This means that a string value associated with 'homepage')\
            ***(  should be interpreted as an identifier that is an IRI )\
            }\
        }\
} .
print(result)
assert('result Context:' in result)

red {     "@context" : {         "name" : "http://schema.org/name",         ***(  This means that 'name' is shorthand for 'http://schema.org/name')         "image" : {             "@id" : "http://schema.org/image",             ***(  This means that 'image' is shorthand for 'http://schema.org/image')             "@type" : "@id"             ***( This means that a string value associated with 'image')             ***( should be interpreted as an identifier that is an IRI)             },         "homepage" : {             "@id" : "http://schema.org/url",             ***( This means that 'homepage' is shorthand for 'http://schema.org/url')             "@type" : "@id"             ***(  This means that a string value associated with 'homepage')             ***(  should be interpreted as an identifier that is an IRI )             }         } } .
reduce in CONTEXT : {"@context" : {("homepage" : {"@id" :
    "http://schema.org/url", "@type" : "@id"}, "image" : {"@id" :
    "http://schema.org/ima

### Identifiers 
[JSON-LD 1.1 / Node Objects](https://www.w3.org/TR/json-ld11/#node-objects)

In [35]:
%%maude
fmod IDENTIFIER is
    protecting STRING .
    protecting IRI .
    sorts BNI Compact-IRI Identifier .
    subsorts IRI Compact-IRI BNI < Identifier < String .
    var S : String .    
    cmb S : IRI  if find(S,":",0) :: Nat .    
endfm    
        

79 (lines sent before.)
--------------------------------------------
fmod IDENTIFIER is
 80 > protecting STRING .
 81 > protecting IRI .
 82 > sorts BNI Compact-IRI Identifier .
 83 > subsorts IRI Compact-IRI BNI < Identifier < String .
 84 > var S : String .
 85 > cmb S : IRI  if find(S,":",0) :: Nat .
 86 > endfm
Maude>



In [36]:
result = %maude red "friends:perico" .
print(result)
assert("result IRI:" in result)

red "friends:perico" .
reduce in IDENTIFIER : "friends:perico" .
rewrites: 3 in 0ms cpu (0ms real) (~ rewrites/second)
result IRI: "friends:perico"
Maude> 


In [37]:
%%maude 
fmod NODE-OBJECT-DECL is 
sort Node-Object .
endfm     

87 (lines sent before.)
--------------------------------------------
fmod NODE-OBJECT-DECL is
 88 > sort Node-Object .
 89 > endfm
Maude>



### Graph 

In [38]:
%%maude
fmod GRAPH is 
    extending NODE-OBJECT-DECL .
    extending JSON *( sort Object to Json-Object ) .

    sorts Graph Named-Graph .
    subsorts Node-Object Named-Graph < Graph < Array .    
    subsort  Named-Graph < Json-Object .
    
    mb null : Graph .
    vars NObj NObj' NObj'' : Node-Object . var ... : List{Json} .  
    var V : Value .  var .... : Set{Entry} . 
    var Ng : Named-Graph .
     mb { "@id" : V, .... } : Named-Graph .
    cmb [ NObj , ... ] : Graph if  [ ... ] :: Graph .
endfm     

90 (lines sent before.)
--------------------------------------------
fmod GRAPH is
 91 > extending NODE-OBJECT-DECL .
 92 > extending JSON *( sort Object to Json-Object ) .
 93 >
 94 > sorts Graph Named-Graph .
 95 > subsorts Node-Object Named-Graph < Graph < Array .
 96 > subsort  Named-Graph < Json-Object .
 97 >
 98 > mb null : Graph .
 99 > vars NObj NObj' NObj'' : Node-Object . var ... : List{Json} .
100 > var V : Value .  var .... : Set{Entry} .
101 > var Ng : Named-Graph .
102 > mb { "@id" : V, .... } : Named-Graph .
103 > cmb [ NObj , ... ] : Graph if  [ ... ] :: Graph .
104 > endfm
Maude>



In [39]:
result = %maude red "@id" : "http://www.schema.org/graph" .
print(result)   
assert('result Entry:' in result)

red "@id" : "http://www.schema.org/graph" .
reduce in GRAPH : "@id" : "http://www.schema.org/graph" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Entry: "@id" : "http://www.schema.org/graph"
Maude> 


In [40]:
result = %maude red .... :: Set{Entry} .
print(result)
assert('result Bool: true' in result)

red .... :: Set{Entry} .
reduce in GRAPH : .... :: Set{Entry} .
rewrites: 1 in 0ms cpu (0ms real) (~ rewrites/second)
result Bool: true
Maude> 


In [41]:
result = %maude red Ng :: Json .
print(result)
assert('result Bool: true' in result)

red Ng :: Json .
reduce in GRAPH : Ng :: Json .
rewrites: 1 in 0ms cpu (0ms real) (~ rewrites/second)
result Bool: true
Maude> 


In [42]:
result = %maude red { "@id" : "http://www.schema.org/graph", .... } .
print(result)
assert('result Named-Graph:' in result)                      

red { "@id" : "http://www.schema.org/graph", .... } .
reduce in GRAPH : {...., "@id" : "http://www.schema.org/graph"} .
rewrites: 1 in 0ms cpu (0ms real) (~ rewrites/second)
result Named-Graph: {...., "@id" : "http://www.schema.org/graph"}
Maude> 


In [43]:
result = %maude red [] . 
print(result)
assert('Graph: null' in result) 
result = %maude red [NObj] . 
print(result)
assert('result Graph: [NObj]' in result) 
result = %maude red [NObj, NObj' , NObj'' ] . 
print(result)
assert("result Graph: [NObj, NObj', NObj'']" in result) 

red [] .
reduce in GRAPH : [] .
rewrites: 2 in 0ms cpu (0ms real) (~ rewrites/second)
result Graph: null
Maude> 
red [NObj] .
reduce in GRAPH : [NObj] .
rewrites: 4 in 0ms cpu (0ms real) (~ rewrites/second)
result Graph: [NObj]
Maude> 
red [NObj, NObj' , NObj'' ] .
reduce in GRAPH : [NObj, NObj', NObj''] .
rewrites: 8 in 0ms cpu (0ms real) (~ rewrites/second)
result Graph: [NObj, NObj', NObj'']
Maude> 


### Reverse map
[JSON-LD 1.1 / Reverse Properties](https://www.w3.org/TR/json-ld11/#reverse-properties)

In [44]:
%%maude
fmod REVERSE-MAP is 
    protecting IDENTIFIER .
    extending NODE-OBJECT-DECL .
    extending JSON *( sort Object to Json-Object ) .

    sorts ReverseMap ReverseId ReverseIdArray .
    subsorts Identifier Node-Object ReverseIdArray  < ReverseId < Value .
    subsort ReverseIdArray  < Array .
    subsort ReverseMap < Json-Object .    


    var Rid : ReverseId . var .... : List{Json} .    
    cmb [ Rid , .... ] : ReverseIdArray  if [ .... ] :: ReverseIdArray .   


    var ReverseRelationIri : IRI . var ReverseRelationSource : ReverseEntryValue . var ... : NeSet{Entry} . 
     mb { ReverseRelationIri : DirectedRelationIri }   : ReverseMap .    
     mb { ReverseRelationIri : ReverseRelationDomain } : ReverseMap .       
    cmb { ReverseRelationIri : Rv , ... } : ReverseMap if  { ... } :: ReverseMap .
        
endfm     

105 (lines sent before.)
--------------------------------------------
fmod REVERSE-MAP is
106 > protecting IDENTIFIER .
107 > extending NODE-OBJECT-DECL .
108 > extending JSON *( sort Object to Json-Object ) .
109 >
110 > sorts ReverseMap ReverseEntryValue ReverseIdArray .
111 > subsorts Identifier Node-Object ReverseIdArray  < ReverseEntryValue < Value .
112 > subsort ReverseIdArray  < Array .
113 > subsort ReverseMap < Json-Object .
114 >
115 >
116 > var Rid : ReverseId . var .... : List{Json} .
117 > cmb [ Rid , .... ] : ReverseIdArray  if [ .... ] :: ReverseIdArray .
118 >
119 >
120 > var Iri : IRI . var Rv : ReverseEntryValue . var ... : NeSet{Entry} .
121 > mb { Iri : Rv } : ReverseMap .
122 > cmb { Iri : Rv , ... } : ReverseMap if  { ... } :: ReverseMap .
123 >
124 > endfm
ReverseId.
Rid:
cmb [ Rid <---*HERE*
cmb [Rid, ....] : ReverseIdArray if [....] :: ReverseIdArray .
Maude>



In [45]:
result = %maude red  "http://perico.com"  . 
print(result)
assert(True) 
result = %maude red  "http://perico.com" :: ReverseEntryValue . 
print(result)
assert(True) 
result = %maude red  { "http://perico.com" : "http://perico.com" } :: ReverseMap . 
print(result)
assert('result Bool: true' in result) 
result = %maude red  { "http://perico.com" : "http://perico.com", \
                       "http://perico.com" : "http://perico.com" } :: ReverseMap . 
print(result)
assert('result Bool: true' in result) 
result = %maude red  { "http://perico.com" : "http://perico.com", \
                       "http://perico.com" : ["http://perico.com","http://perico.com"] } :: ReverseMap . 
print(result)
assert('result Bool: true' in result) 


red  "http://perico.com"  .
reduce in REVERSE-MAP : "http://perico.com" .
rewrites: 3 in 0ms cpu (0ms real) (~ rewrites/second)
result IRI: "http://perico.com"
Maude> 
red  "http://perico.com" :: ReverseEntryValue .
reduce in REVERSE-MAP : "http://perico.com" :: ReverseEntryValue .
rewrites: 4 in 0ms cpu (0ms real) (~ rewrites/second)
result Bool: true
Maude> 
red  { "http://perico.com" : "http://perico.com" } :: ReverseMap .
reduce in REVERSE-MAP : {"http://perico.com" : "http://perico.com"} ::
    ReverseMap .
rewrites: 5 in 0ms cpu (0ms real) (~ rewrites/second)
result Bool: true
Maude> 
red  { "http://perico.com" : "http://perico.com",                         "http://perico.com" : "http://perico.com" } :: ReverseMap .
reduce in REVERSE-MAP : {"http://perico.com" : "http://perico.com",
    "http://perico.com" : "http://perico.com"} :: ReverseMap .
rewrites: 6 in 0ms cpu (0ms real) (~ rewrites/second)
result Bool: true
Maude> 
red  { "http://perico.com" : "http://perico.com",        

AssertionError: 

In [ ]:
result = %maude red \
{\
    "http://example.com/vocab#parent" : [\
      {\
        "@id" : "#bart",\
        "http://example.com/vocab#name" : "Bart"\
      }, {\
        "@id" : "#lisa",\
        "http://example.com/vocab#name" : "Lisa"\
      }\
    ]\
  } :: ReverseMap .
print(result)
assert('result Bool: true' in result)       

### Node Object 
[JSON-LD 1.1 / Node Objects](https://www.w3.org/TR/json-ld11/#node-objects)

In [ ]:
%%maude
fmod NODE-OBJECT is
protecting KEYWORD .
protecting IDENTIFIER .
protecting JSON *( sort Object to Json-Object ) .

    

sorts Node-Object ^Node-Object .    
subsorts Node-Object ^Node-Object < Object .

sort Value .
    
var O : Object . var  N : Node-Object .
var V : Value .    
cmb O : ^Node-Object if
        ({ "@context"  : V  , ...} := O and not V :: Context         )    
     or ({ "@id"       : V  , ...} := O and not V :: Identifier      )
     or ({ "@graph"    : V  , ...} := O and not V :: Graph           )
     or ({ "@type "    : V  , ...} := O and not V :: Identifier      )
     or ({ "@reverse"  : V  , ...} := O and not V :: Reverse-Map     ) 
     or ({ "@included" : V  , ...} := O and not V :: Included-Block  )
     or ({ "@index"    : V  , ...} := O and not V :: String          )
     or ({ "@nest "    : V  , ...} := O and not V :: Property-Nesting) .

cmb O : Node-Object if not O :: ^Node-Object .
endfm     
    

### Expand

### Terms

### JSON-LD BASIC

In [ ]:
%%maude 
show modules .

### JSON-LD_TERM

### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB